In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
batch_size = 256
learning_rate=1e-3
epochs =15

In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.Normalize((0.5), (0.5))
                                ])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
                                ])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



In [ ]:
classes =[
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

In [ ]:
def draw_data(img, lab, it_subplot):
  # img tensor --> numpy
  img = img.numpy()
  # c h w --> plt : h w c
  img = img.transpose([1, 2, 0])
  # -1~1 --> 0~1
  img = img / 2 + 0.5
  title_ = 'gt [%s]' % (classes[lab])
  plt.subplot(1, 3, it_subplot)
  plt.title(title_)
  plt.imshow(img, 'gray')
  

In [ ]:
imgs, labels = next(iter(trainloader))

In [ ]:
imgs.shape, labels.shape

(torch.Size([256, 1, 28, 28]), torch.Size([256]))

it_sample = 0
for img, lab in zip(imgs, labels):
  draw_data(img, lab, it_sample%3+1)
  
  if (it_sample%3 == 0) and (it_sample != 0):
    plt.show()
  it_sample += 1

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.name = "SimpleCNN"
        self.conv_layer = nn.Sequential(
            # 28 *28 *1 784
            # 100-->
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3)),            
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3)),
            nn.ReLU(inplace=True),
            
            # nn.MaxPool2d(kernel_size=2, stride=2), # subsampling
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3)),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p = 0.5),

            nn.Linear(3200, 512),
            nn.ReLU(inplace=True),

            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128, 10)
        )
     

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(-1, 3200)
        x = self.fc_layer(x)
        return x



In [ ]:
import torchsummary
model = SimpleCNN()
torchsummary.summary(model, (1, 28, 28), device = 'cpu')
model = model.to(device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,640
              ReLU-4           [-1, 32, 24, 24]               0
            Conv2d-5           [-1, 32, 22, 22]           9,248
              ReLU-6           [-1, 32, 22, 22]               0
            Conv2d-7           [-1, 32, 20, 20]           9,248
              ReLU-8           [-1, 32, 20, 20]               0
         MaxPool2d-9           [-1, 32, 10, 10]               0
          Dropout-10                 [-1, 3200]               0
           Linear-11                  [-1, 512]       1,638,912
             ReLU-12                  [-1, 512]               0
           Linear-13                  [-1, 128]          65,664
             ReLU-14                  [

In [ ]:
def test_accuracy(model, test_dataloader, batch_size):
  
    with torch.no_grad():
        model.eval()
        total_correct =0
        total_cnt = 0

        for it_batch, data in enumerate(test_dataloader) :
            imges, labels = data

            imges = imges.to(device)
            # gt --> 확률벡터로 변환
            labels = torch.eye(10)[labels]  
            labels = labels.to(device)

            preds = model(imges)

            t1 = torch.argmax(labels, dim=1)
            t2 = torch.argmax(preds, dim=1)

            total_correct += (t1==t2).sum()
            total_cnt +=1

    # prt('correct [%d] acc [%.2f] ', total_correct, total_correct/(total_cnt*batch_size))

    return total_correct


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

for it_epochs in range(epochs):
    train_loss = 0
    loss_cnt = 0
    for it_batch, data in tqdm(enumerate(trainloader)):

        model.train()
        imges, labels = data

        imges = imges.to(device)
        labels = torch.eye(10)[labels]  # 확률벡터로 변환
        labels = labels.to(device)

        preds = model(imges)

        optimizer.zero_grad()

        loss = criterion(preds, labels)
        train_loss += loss.item()
        loss_cnt += 1

        loss.backward()
        optimizer.step()

    prt = 'epoch %d train loss: %.2f' % (it_epochs, train_loss/loss_cnt)
    print(prt)
    total_cnt = len(testset)
    total_correct = test_accuracy(model, testloader, batch_size)
    prt = 'correct [%d] ##### acc [%.2f] %d' % (total_correct, total_correct/total_cnt, total_cnt)
    print(prt)


0it [00:00, ?it/s]

epoch 0 train loss: 0.10
correct [9275] ##### acc [0.93] 10000


0it [00:00, ?it/s]

epoch 1 train loss: 0.09
correct [9251] ##### acc [0.93] 10000


0it [00:00, ?it/s]

...

epoch 12 train loss: 0.07
correct [9245] ##### acc [0.92] 10000


0it [00:00, ?it/s]

epoch 13 train loss: 0.07
correct [9262] ##### acc [0.93] 10000


0it [00:00, ?it/s]

epoch 14 train loss: 0.07
correct [9261] ##### acc [0.93] 10000


In [ ]:
len(testset)

10000